In [136]:
import tensorflow as tf
import numpy as np
import pandas as pd

items = pd.read_csv(r"..\Data\Raw\items.csv", delimiter=",")
print(items.head())
print(items.shape)
items = items.drop_duplicates(subset=['item_name'], keep=False)
print(items.shape)

                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
(22170, 3)
(22170, 3)


In [137]:
test = pd.read_csv(r"..\Data\Raw\test.csv", delimiter=",")
print(test.head())
print(test['shop_id'].nunique())
print(test['ID'].nunique())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
42
214200


In [138]:
sales = pd.read_csv(r"..\Data\Raw\sales_train.csv", delimiter=",")
print(sales.head())
print(sales.shape)
sales = sales.drop_duplicates(subset=['date', 'shop_id', 'item_id'], keep=False)
print(sales.shape)
sales = sales[sales.item_cnt_day<1000]
sales = sales[sales.item_price<60000]
sales = sales[sales.item_price>0]
print(sales.shape)
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales_monthly = sales.groupby(["date_block_num", "shop_id", "item_id"]).agg({'item_cnt_day': ['sum'], 'item_price':['median'], 'revenue':['sum']})
print(sales_monthly.head(20))
sales_monthly.sample(10)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
(2935849, 6)
(2935793, 6)
(2935789, 6)
                               item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              0       32               6.0      221.0  1326.0
                       33               3.0      347.0  1041.0
                       35               1.0      247.0   247.0
                       43               1.0      221.0   221.0
                       51               2.0      128.5   257.0
             

item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
4              50      6315             1.0      199.0   199.0
19             58      19021            1.0      399.0   399.0
               28      21427            4.0      149.0   596.0
25             47      4993             1.0     1998.0  1998.0
9              4       7303             1.0      299.0   299.0
16             46      7895             1.0      749.0   749.0
4              37      9971             1.0      149.0   149.0
0              16      3223             2.0      449.0   898.0
24             31      13855            1.0     1199.0  1199.0
               27      14280            1.0      349.0   349.0

In [139]:
#join to test set to get only necessary IDs
month_range = range(1, 34)
month_df = pd.DataFrame(month_range)

month_df.rename(columns={0 :'date_block_num'}, inplace=True )
print(month_df.head())
month_df['temp'] = 1
test['temp'] = 1
test_monthly = test.merge(month_df, on=['temp'])
test_monthly.drop(['temp'], axis=1, inplace=True)
whole_df = test_monthly.merge(sales_monthly, how='left', on=['date_block_num', 'item_id', 'shop_id'])
whole_df.sample(10)
print(whole_df.shape)

   date_block_num
0               1
1               2
2               3
3               4
4               5
(7068600, 7)


e:\latest pycharm\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [140]:
categories = pd.read_csv(r"..\Data\Raw\item_categories.csv", delimiter=",")
print(categories.head())


        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4


In [141]:
print(items.shape)
print(categories.shape)
item_cats = items.merge(categories, how='left', on=['item_category_id'])
print(item_cats.shape)
print(item_cats.head())
item_cats.drop(['item_category_name', 'item_name'], axis=1, inplace=True)
print(item_cats.head())
print(item_cats['item_id'].nunique())
print(item_cats.shape)
whole_df = whole_df.merge(item_cats, how='left', on=['item_id'])
whole_df.sample(10)

(22170, 3)
(84, 2)
(22170, 4)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  
0                40                            Кино - DVD  
1                76  Программы - Для дома и офиса (Цифра)  
2                40                            Кино - DVD  
3                40                            Кино - DVD  
4                40                            Кино - DVD  
   item_id  item_category_id
0        0                40
1        1                76
2        2                40
3        3                40
4        4                40
22170
(22170, 2)


,ID,shop_id,item_id,date_block_num,"(item_cnt_day, sum)","(item_price, median)","(revenue, sum)",item_category_id
4686738,142022,57,9524,13,NaN,NaN,NaN,31
5220773,158205,56,3831,9,NaN,NaN,NaN,55
4464273,135281,48,19478,1,NaN,NaN,NaN,61
5495730,166537,36,12463,10,NaN,NaN,NaN,55
1802576,54623,26,8257,18,NaN,NaN,NaN,40
2500660,75777,21,4954,20,NaN,NaN,NaN,76
1489642,45140,28,11874,23,NaN,NaN,NaN,54
1690917,51239,26,7863,31,1.0,1299.0,1299.0,28
581000,17606,3,9206,3,NaN,NaN,NaN,61
283612,8594,4,6408,11,NaN,NaN,NaN,24


In [142]:
whole_df.fillna(0, inplace=True)
list(whole_df.columns)
whole_df.rename(columns={('item_cnt_day', 'sum'):'total_count', ('item_price', 'median'): 'median_price', ('revenue', 'sum'):'total_revenue'}, inplace=True )
whole_df['total_count'].clip(0,20)
print(whole_df.sample(10))
whole_df.to_csv('../Data/Processed/pre_normalised_whole_df.csv', index=False)

             ID  shop_id  item_id  date_block_num  total_count  median_price  \
1684438   51043       26     5822              20         23.0        1049.0   
6019820  182418       38     4104              27          0.0           0.0   
4240787  128508       47     4349              24          0.0           0.0   
862160    26126        7     3953               3          0.0           0.0   
2796031   84728       16    16494               8          0.0           0.0   
286728     8688        4    11324              25          0.0           0.0   
3740051  113334       49     3605              30          0.0           0.0   
3557987  107817       50    13708              27          0.0           0.0   
2579131   78155       15    17655              17          0.0           0.0   
2325482   70469       24     7342               6          0.0           0.0   

         total_revenue  item_category_id  
1684438        24127.0                35  
6019820            0.0           

In [157]:
whole_df.sort_values(by=['ID', 'date_block_num'])
print(whole_df.head(34))
whole_df_arr = whole_df.values
print(whole_df_arr.shape)
whole_df_arr = whole_df_arr.reshape(214200, 33, 8)
print(whole_df_arr.shape)
print(whole_df_arr[:1,19:20])

    ID  shop_id  item_id  date_block_num  total_count  median_price  \
0    0        5     5037               1          0.0           0.0   
1    0        5     5037               2          0.0           0.0   
2    0        5     5037               3          0.0           0.0   
3    0        5     5037               4          0.0           0.0   
4    0        5     5037               5          0.0           0.0   
5    0        5     5037               6          0.0           0.0   
6    0        5     5037               7          0.0           0.0   
7    0        5     5037               8          0.0           0.0   
8    0        5     5037               9          0.0           0.0   
9    0        5     5037              10          0.0           0.0   
10   0        5     5037              11          0.0           0.0   
11   0        5     5037              12          0.0           0.0   
12   0        5     5037              13          0.0           0.0   
13   0

In [159]:
#prepare training and validation sets
train_arr = whole_df_arr[:, :26]
val_arr = whole_df_arr[:, 26:]
print(train_arr.shape)
print(val_arr.shape)

(214200, 26, 8)
(214200, 7, 8)


In [ ]:
#prepare training and validation sets
n = len(whole_df)
train_df = whole_df[:int(n*0.8)]
val_df = whole_df[int(n*0.2):]
num_features = whole_df.shape[1]

#normalise data
train_mean = train_df.mean()
train_std = train_df.std()
train_df = (train_df - train_mean)/train_std
val_df = (val_df - train_mean)/train_std

train_df.to_csv('../Data/Processed/normalised_training_df.csv', index=False)
val_df.to_csv('../Data/Processed/normalised_val_df.csv', index=False)

In [ ]:
print(train_df.shape)